## Binder as Dataverse External Tool

This Binder notebook demonstrates one possible approach to Dataverse External Tools integration.

See the Dataverse [External Tools](http://guides.dataverse.org/en/latest/installation/external-tools.html) documentation for additional details.  In short, Dataverse adds an "Explore" button next to tabular data. When clicked, the `fileId` and `siteUrl` are passed to the remote service.

The Dataverse Data Explorer provides an example of an external tool.  To see it in action:
* Go to https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/SCN9LA
* Select "Explore" next to `thalpbblackaff.tab`
* You'll be redirected to:  https://scholarsportal.github.io/Dataverse-Data-Explorer/?fileId=2865473&siteUrl=https://dataverse.harvard.edu

A few notes:
* [iqss/dataverse#5028](https://github.com/IQSS/dataverse/issues/5028) will add support for dataset-level external tools
* It would also be possible to just read in the full dataset, just not as an integrated external tool.
* The example dataset used below actually contains R scripts, but I'm ignoring them for now and just focusing on the data file.

In [ ]:
import requests
import pandas as pd
from xml.etree import ElementTree
from IPython.display import display, Markdown

Use Javascript to read the query string parameters. Inspired by [CINERGI] (happening in custom.js)

In [ ]:
try:
    print("Parsed query resulted in:")
    print(" - meta_url = " + meta_url)
    print(" - data_url = " + data_url)
except NameError:
    print("Parsing query failed. Saaaaaad....")
    print("Did you restart the kernel without refreshing the page? That would do it...")
    data_url = 'https://dataverse.harvard.edu/api/access/datafile/2865473'
    meta_url = data_url + '/metadata/ddi'

Get the data package metadata (stolen from @xarthisius's [WIP](https://github.com/whole-tale/girder_wholetale/pull/175))

In [ ]:
resp = requests.get(meta_url)
tree = ElementTree.fromstring(resp.text)

ddsc = tree.find('{http://www.icpsr.umich.edu/DDI}stdyDscr')
citation = ddsc.find('{http://www.icpsr.umich.edu/DDI}citation')
title_stmt = citation.find('{http://www.icpsr.umich.edu/DDI}titlStmt')
title = title_stmt.find('{http://www.icpsr.umich.edu/DDI}titl').text
display(Markdown('## %s' % title ))

Now get the data and read into Pandas for display

In [ ]:
df = pd.read_csv(data_url, delimiter = '\\t')

In [ ]:
df.head()

This is just a proof-of-concept and leaves much to be desired.  